# Etude du DF total

## Importer les librairies

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re


### Obtenir l'info complète du DF

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

## Importer le DF

In [2]:
df= pd.read_csv("gaming_total.csv", index_col=0)
df.head()

C:\Users\debor\AppData\Local\Temp\ipykernel_47552\60888040.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv("gaming_total.csv", index_col=0)


,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,...,user_review,Rank,test_note_JVC,avis_count_JVC,avis_note_JVC,Classification_Age_JVC,max_joueurs_JVC,Mode_JVC,Support_JVC,Langue_JVC
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,...,8.1,1.0,14,(687),13.8,+7 ans\r,Jusqu'à 4 joueurs\r,NaN,DVD\r,Voix en français Textes en français
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,...,NaN,NaN,19,(3),19,+3 ans\r,NaN,Jouable en solo\r,eShop Console Virtuelle Cartouche\r,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82,...,8.4,3.0,16,(3023),17.4,+3 ans\r,"4 joueurs en écran partagé, 12 joueurs online\r",Multi sur le même écran Multi en ligne\r,DVD\r,NaN
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00,...,8.2,4.0,15,(581),16.5,+3 ans\r,Jusqu'à 4 joueurs\r,NaN,DVD\r,Voix en français Textes en français\r
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,...,NaN,NaN,no_data,no_data,no_data,NaN,NaN,NaN,NaN,NaN


In [3]:
df = df.replace('no_data', np.nan)

In [4]:
# définir une expression régulière pour extraire les 2 premiers mots qui suivent "voix"
pattern = re.compile(r'\bvoix\W+(\w+\W+){0,1}\w+\b', flags=re.IGNORECASE)

# extraire les 2 premiers mots qui suivent "voix" pour chaque chaîne de caractères
df['langue_parlée_JVC'] = df['Langue_JVC'].apply(lambda x: pattern.search(str(x)).group(0) if pattern.search(str(x)) else None)

# définir une expression régulière pour extraire les 2 premiers mots qui suivent "Textes"
pattern = re.compile(r'\btextes\W+(\w+\W+){0,1}\w+\b', flags=re.IGNORECASE)

# extraire les 2 premiers mots qui suivent "Textes" pour chaque chaîne de caractères
df['texte_JVC'] = df['Langue_JVC'].apply(lambda x: pattern.search(str(x)).group(0) if pattern.search(str(x)) else None)


In [5]:
# fonction pour extraire le premier chiffre d'une chaîne de caractères
def extract_first_number(s):
    match = re.search(r'\d+', s)
    if match:
        return int(match.group())
    else:
        return None

# appliquer la fonction à la colonne 'max_joueurs' pour extraire le premier chiffre
df['max_joueurs_JVC'] = df['max_joueurs_JVC'].apply(lambda x: extract_first_number(str(x)))
df['nb_joueurs_JVC'] = df['max_joueurs_JVC'].apply(lambda x: extract_first_number(str(x)))
df['PEGI_goo'] = df['PEGI_goo'].apply(lambda x: extract_first_number(str(x)))
df['Classification_Age_JVC'] = df['Classification_Age_JVC'].apply(lambda x: extract_first_number(str(x)))
df['PEGI_wiki'] = df['PEGI_wiki'].apply(lambda x: extract_first_number(str(x)))

In [6]:
df['avis_count_JVC']= df['avis_count_JVC'].apply(lambda x: x.replace('(', '').replace(')', '') if isinstance(x, str) else x)
df['Mode_JVC']= df['Mode_JVC'].apply(lambda x: x.replace('\r', '') if isinstance(x, str) else x)
df['Support_JVC']= df['Support_JVC'].apply(lambda x: x.replace('\r', '') if isinstance(x, str) else x)
df['game_mode_wiki']= df['game_mode_wiki'].apply(lambda x: x.replace('\r', '').replace('\n', '') if isinstance(x, str) else x)

In [7]:
# extraire la langue
for i, row in df.iterrows():
    if not pd.isnull(row['langue_parlée_JVC']):
        df.at[i, 'langue_parlée_JVC'] = row['langue_parlée_JVC'].split()[2]

In [8]:
# extraire la langue du texte
for i, row in df.iterrows():
    if not pd.isnull(row['texte_JVC']):
        df.at[i, 'texte_JVC'] = row['texte_JVC'].split()[2]

In [9]:
df= df.drop('Langue_JVC', axis=1)

In [10]:
#df=df.drop(['Rank_x', 'Rank_y'], axis=1)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16628 entries, 0 to 16630
Data columns (total 38 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Name                    16628 non-null  object 
 1   Platform                16628 non-null  object 
 2   Year                    16358 non-null  float64
 3   Genre                   16628 non-null  object 
 4   Publisher               16570 non-null  object 
 5   NA_Sales                16628 non-null  float64
 6   EU_Sales                16628 non-null  float64
 7   JP_Sales                16628 non-null  float64
 8   Other_Sales             16628 non-null  float64
 9   Global_Sales            16628 non-null  float64
 10  developpeur_goo         1500 non-null   object 
 11  distributeur_goo        14 non-null     object 
 12  langue_goo              75 non-null     object 
 13  PEGI_goo                486 non-null    float64
 14  game_series_goo         524 non-null  

In [11]:
df.to_csv('gaming_total_v2.csv')

In [12]:
df.describe(include='all')

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,...,test_note_JVC,avis_count_JVC,avis_note_JVC,Classification_Age_JVC,max_joueurs_JVC,Mode_JVC,Support_JVC,langue_parlée_JVC,texte_JVC,nb_joueurs_JVC
count,16628,16628,16358.000000,16628,16570,16628.000000,16628.000000,16628.000000,16628.000000,16628.000000,...,10032,9133,9133,7111.000000,2566.000000,3933,7369,3962,4616,2566.000000
unique,11493,31,NaN,12,578,NaN,NaN,NaN,NaN,NaN,...,20,546,182,NaN,NaN,44,531,3,3,NaN
top,Need for Speed: Most Wanted,PS2,NaN,Action,Electronic Arts,NaN,NaN,NaN,NaN,NaN,...,--,0,--,NaN,NaN,Jouable en solo,DVD,français,français,NaN
freq,12,2166,NaN,3333,1357,NaN,NaN,NaN,NaN,NaN,...,3856,1141,1141,NaN,NaN,2304,1848,2554,3841,NaN
mean,NaN,NaN,2006.410136,NaN,NaN,0.264508,0.146513,0.077641,0.048037,0.536973,...,NaN,NaN,NaN,8.637041,3.056898,NaN,NaN,NaN,NaN,3.056898
std,NaN,NaN,5.825095,NaN,NaN,0.816007,0.504917,0.309029,0.188426,1.553710,...,NaN,NaN,NaN,5.716143,6.034071,NaN,NaN,NaN,NaN,6.034071
min,NaN,NaN,1980.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.010000,...,NaN,NaN,NaN,3.000000,1.000000,NaN,NaN,NaN,NaN,1.000000
25%,NaN,NaN,2003.000000,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.060000,...,NaN,NaN,NaN,3.000000,2.000000,NaN,NaN,NaN,NaN,2.000000
50%,NaN,NaN,2007.000000,NaN,NaN,0.080000,0.020000,0.000000,0.010000,0.170000,...,NaN,NaN,NaN,7.000000,2.000000,NaN,NaN,NaN,NaN,2.000000
75%,NaN,NaN,2010.000000,NaN,NaN,0.240000,0.110000,0.040000,0.040000,0.470000,...,NaN,NaN,NaN,12.000000,2.000000,NaN,NaN,NaN,NaN,2.000000
